In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib editdistance
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [26]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import json
from datasets import load_dataset, Data
import numpy as np
from tqdm import tqdm

from utils import similar_tag

In [ ]:
torch.cuda.is_available()

In [2]:
device = "cuda"
peft_model_id = "ooferdoodles/text2tag"
base_model = "facebook/opt-350m"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    load_in_8bit=True,
    device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(base_model)

model = PeftModel.from_pretrained(
    model, peft_model_id, torch_dtype=torch.float16)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


In [37]:
data = json.load(open(r"dataset/test_data.json"))
tag_dict = similar_tag.load_dict()

NameError: name 'Dataset' is not defined

In [48]:
generation_config = GenerationConfig(
    temperature=1,
    top_p=0.75,
    top_k=40,
    num_beams=1,
    typical_p=1,
    do_sample=False,
    max_new_tokens=300,
    no_repeat_ngram_size=2,
#     truncation_length=2048,
#     min_length=0,
#     add_bos_token=True,
#     ban_eos_token=False,
#     skip_special_tokens=True,
#     stopping_strings=[],
#     penalty_alpha=0,
#     repetition_penalty=1.2,
#     encoder_repetition_penalty=1,
)
max_token_scale = 0.3

In [49]:
accuracy_list = []
caption_list = []
pred_list = []

pipe = pipeline("text-generation",
                model=model,
                device=0,
                tokenizer=tokenizer,
                max_new_tokens=max_new_tokens,
                config=generation_config)

prompts = [f"### Caption: {x['caption_string']}\n### Tags: " for x in data]
preds = pipe(prompts[:5])
corrected_preds = (similar_tag.correct_tags(
    x[0]['generated_text'].split("### Tags:")[-1]) for x in preds)

# for data_point in tqdm(data[:5]):
#     caption = data_point["caption_string"]
#     tags = data_point["tag_string"].split(", ")
#     caption_list.append(caption)

#     prompt = f"### Caption: {caption}\n### Tags: "

#     preds = pipe(prompt)[0]['generated_text'].split('### Tags')[-1]

#     corrected_preds = similar_tag.correct_tags(preds.split(", "), tag_dict)
#     pred_list.append(corrected_preds)

#     correct_count = len(set(tags).intersection(corrected_preds))
#     incorrect_count = len(tags) - correct_count
#     accuracy = correct_count / len(tags) * 100
#     accuracy_list.append(accuracy)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForCausalLM', 'RoCBer

[[{'generated_text': '### Caption: With her mesmerizing gaze and ethereal wings, Clair Lasbard, from the Star Ocean franchise, comes to life in monochrome hues thanks to the exquisite artistry of kiikii_(kitsukedokoro). Her alluring smile and intricate horns, complemented by her long hair and gloves, make for a stunning portrayal of this character known for her large breasts. Amidst a simple white background, this piece captures the beauty and mystique of Clair, leaving us entranced by her presence.\n### Tags: ikii_(kitsukedokoro), breasts, gloves, horn, large_breasts, monochrome, multiple_girls, star_oregon, star_oregon_2, solo, horn, white_background, white_'}],
 [{'generated_text': "### Caption: Kisaragi Ai's art features Garma Zabi, a young boy with short hair and brown eyes adjusting his hair and bowtie while sporting a flower and a smile in a purple-eyed, formal solo pose with a suit, vest, and bow.\n### Tags: izanami_kazuya, blush, bow, bowtie, flower, bowtie, hair_bow, hair_orn

In [ ]:
#     inputs = tokenizer(prompt, return_tensors='pt')
#     input_ids = inputs["input_ids"].to(device)
    
#     max_new_tokens = int(len(caption) * max_token_scale)

# #     output_tokens = model.generate(
# #         input_ids=input_ids,
# #         generation_config=generation_config,
# #         max_new_tokens=max_new_tokens)
    
#     preds = tokenizer.decode(output_tokens[0], skip_special_tokens=True).split('### Tags:')[-1]

In [8]:
print(f"Accuracy: {sum(accuracy_list)/len(accuracy_list)}")
for i in range(3):
    print(f"Caption: {caption_list[i]}")
    print(f"Preds: {pred_list[i]}")
    print(f"Correct: {data[i]['tag_string'].split(', ')}")
    print()

Accuracy: 24.69228074603999
Caption: With her mesmerizing gaze and ethereal wings, Clair Lasbard, from the Star Ocean franchise, comes to life in monochrome hues thanks to the exquisite artistry of kiikii_(kitsukedokoro). Her alluring smile and intricate horns, complemented by her long hair and gloves, make for a stunning portrayal of this character known for her large breasts. Amidst a simple white background, this piece captures the beauty and mystique of Clair, leaving us entranced by her presence.
Preds: ['black_hair', 'breasts', 'corn', 'gloves', 'hibiki_(zerocodo)', 'large_breasts', 'monochrome', 'navel', 'open_mouth', 'purple_eyes', 'red_hair', 'solo', 'white_background']
Correct: ['1girl', 'breasts', 'closed_mouth', 'gloves', 'horns', 'large_breasts', 'long_hair', 'looking_at_viewer', 'monochrome', 'simple_background', 'smile', 'solo', 'star_ocean', 'star_ocean_anamnesis', 'star_ocean_till_the_end_of_time', 'white_background', 'wings']

Caption: Kisaragi Ai's art features Garma

In [25]:
output

CausalLMOutputWithPast(loss={'logits': tensor([[[-0.4590, -0.2103,  9.4844,  ..., -1.4004, -0.5713,  1.2080],
         [-3.8105, -6.5703,  5.2305,  ..., -6.3906, -8.8906, -3.2754],
         [-5.9648, -8.5078,  4.6211,  ..., -7.4492, -7.7578, -7.4922],
         ...,
         [ 0.9985, -1.0947, 12.8516,  ...,  0.9795, -2.6816,  3.0332],
         [ 4.7305,  7.2461, 14.1875,  ...,  7.1211,  3.1777,  5.2031],
         [ 5.9102,  5.0312,  0.9146,  ...,  6.1328,  4.5586,  4.0234]]],
       device='cuda:0', dtype=torch.float16), 'past_key_values': ((tensor([[[[ 0.3237, -0.3425, -0.4148,  ...,  0.3088, -0.1377, -0.3411],
          [-0.6230,  0.6367,  0.5649,  ..., -0.5850,  0.6021,  0.6387],
          [-0.7358,  0.7148,  0.6929,  ..., -0.6919,  0.6606,  0.7261],
          ...,
          [-0.4734,  0.4719,  0.4751,  ..., -0.4705,  0.4690,  0.4854],
          [-0.4929,  0.4778,  0.5054,  ..., -0.5137,  0.5156,  0.4751],
          [-0.3538,  0.3325,  0.3647,  ..., -0.3479,  0.3159,  0.3259]],

   

In [32]:
data_point = data[5]

caption = data_point["caption_string"]
tags = data_point["tag_string"].split(", ")
caption_list.append(caption)

prompt = f"### Caption: {caption}\n### Tags: "
inputs = tokenizer(prompt, return_tensors='pt')
input_ids = inputs["input_ids"].to(device)

max_new_tokens = int(len(caption) * max_token_scale)


pipe = pipeline("text-generation", 
                model=model, 
                device=0, 
                tokenizer=tokenizer, 
                max_new_tokens=max_new_tokens,
                config=generation_config)
pipe(prompt)

# output = model(input_ids)
# tokenizer.decode(output['logits'])
# tokenizer.decode(output)

# output_tokens = model.generate(
#     input_ids=input_ids,
#     generation_config=generation_config,
#     max_new_tokens=max_new_tokens)


# preds = tokenizer.decode(output_tokens[0], 
#                          skip_special_tokens=True).split('### Tags:')[-1]

# print(preds)

# corrected_preds = similar_tag.correct_tags(preds.split(", "), tag_dict)
# print(corrected_preds)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForCausalLM', 'RoCBer

[{'generated_text': '### Caption: The artwork by wenny02 features one girl and three boys from the game League of Legends. The character Ezreal is included in the artwork. The setting is outdoors at a market stall with a building and palm tree in the background. The characters are wearing various clothing items such as fur-trimmed jackets, gloves, knee pads, and hoodies. One of the characters is holding an apple while another has food on their face. There are also various items such as a vase, a box, and a rug included in the artwork.\n### Tags: izreal, apple, food, hood, hoodie, hoodie_over_head, hoodie_over_shoulder, food, food_on_face, food_on_face_in_clothes, gloves, gloves_over_shoulder, gloves_over_shoulder, gloves_over_shoulder, food, food_on_face, food_on_face_in_clothes, food_on_face_in_clothes, food_on_face_in_clothes, food_on_face_in_clothes, food_on_face_in_clothes, food_on_face_in_clothes, food_on_face_in_'}]

In [17]:
input_ids

tensor([[    2, 48134, 19754,    35,    20, 14129,    30,   885, 11867,  4197,
          1575,    65,  1816,     8,   130,  2786,    31,     5,   177,   815,
             9, 24322,     4,    20,  2048, 32050,  8726,    16,  1165,    11,
             5, 14129,     4,    20,  2749,    16, 13384,    23,    10,   210,
         16179,    19,    10,   745,     8, 14262,  3907,    11,     5,  3618,
             4,    20,  3768,    32,  2498,  1337,  5296,  1964,   215,    25,
         15503,    12,  4328,   757,  4567, 20200,     6, 14341,     6,  4117,
         18569,     6,     8,  9659,   918,     4,   509,     9,     5,  3768,
            16,  1826,    41, 15162,   150,   277,    34,   689,    15,    49,
           652,     4,   345,    32,    67,  1337,  1964,   215,    25,    10,
           748,  3175,     6,    10,  2233,     6,     8,    10, 28862,  1165,
            11,     5, 14129,     4, 50118, 48134, 36518,    35,  1437]],
       device='cuda:0')

In [ ]:
data[1]['tag_string']

In [ ]:
pred_list[np.array(accuracy_list).argmin()]